In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import json
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import numpy as np


with open(r"C:\Users\M L A\Desktop\data\data.json", encoding="utf-8") as f:
    data = json.load(f)


questions = [item['input_text'] for item in data['responses']]
answers = [item['response_text'] for item in data['responses']]


label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(answers)


vectorizer = CountVectorizer(max_features=50)  
question_vectors = vectorizer.fit_transform(questions).toarray()


class ChatDataset(Dataset):
    def __init__(self, questions, labels):
        self.questions = questions
        self.labels = labels

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        label = self.labels[idx]
        return question, label


class ChatBotModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ChatBotModel, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        return x


input_size = question_vectors.shape[1] 
hidden_size = 8
output_size = len(set(encoded_labels))
learning_rate = 0.001
num_epochs = 1000

# إعداد بيانات التدريب
train_questions, test_questions, train_labels, test_labels = train_test_split(
    question_vectors, encoded_labels, test_size=0.2, random_state=42
)
train_data = ChatDataset(train_questions, train_labels)
train_loader = DataLoader(train_data, batch_size=2, shuffle=True)


model = ChatBotModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    for question, label in train_loader:
       
        question = torch.tensor(question, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)
        
       
        outputs = model(question)
        loss = criterion(outputs, label)
        
       
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


torch.save(model.state_dict(), r"C:\Users\M L A\Desktop\data\chatbot_model1.pt")
print("تم حفظ النموذج كملف .pt")


def predict_response(question):
    model.eval()
    question_vector = vectorizer.transform([question]).toarray()
    question_tensor = torch.tensor(question_vector, dtype=torch.float32)
    with torch.no_grad():
        output = model(question_tensor)
        _, predicted = torch.max(output, 1)
    return label_encoder.inverse_transform([predicted.item()])[0]


print(predict_response("كيف حالك؟"))
